In [1]:
import pandas as pd
import numpy as np
import os
import pandas_gbq
from google.cloud import bigquery
import glob
import openpyxl

In [2]:
os.chdir('G:\\\\Drives compartilhados\\\\República.org\\\\4. Equipes\\\\Dados e Comunicação\\\\DADOS\\\\415 - Repositório de Dados\\\\Repositório Local')


In [3]:
df = pd.read_excel('MUNIC//Base_MUNIC_2019.xlsx', sheet_name='Direitos humanos', usecols=['COD UF','CodMun','NOME MUNIC','MDHU04','MDHU05','MDHU06','MDHU07'])
df

,CodMun,COD UF,NOME MUNIC,MDHU04,MDHU05,MDHU06,MDHU07
0,1100015,11,Alta Floresta D'Oeste,-,-,-,-
1,1100023,11,Ariquemes,-,-,-,-
2,1100031,11,Cabixi,-,-,-,-
3,1100049,11,Cacoal,Masculino,52,Branca,Ensino superior incompleto
4,1100056,11,Cerejeiras,Feminino,63,Parda,Ensino superior completo
...,...,...,...,...,...,...,...
5565,5222005,52,Vianópolis,Feminino,(*) Não soube informar,Branca,Ensino superior completo
5566,5222054,52,Vicentinópolis,-,-,-,-
5567,5222203,52,Vila Boa,-,-,-,-
5568,5222302,52,Vila Propício,-,-,-,-


In [4]:
cod_uf = pd.read_csv('MUNIC//MUNIC_educacao_2021.csv', sep=';')[['UF','Cod UF']]

In [5]:
x= cod_uf.pivot_table(columns=('UF','Cod UF'), aggfunc='size')


In [6]:
cod_uf = pd.DataFrame(x).reset_index()[['UF','Cod UF']]

In [7]:
df = df.merge(cod_uf, right_on='Cod UF',left_on='COD UF')
df

,CodMun,COD UF,NOME MUNIC,MDHU04,MDHU05,MDHU06,MDHU07,UF,Cod UF
0,1100015,11,Alta Floresta D'Oeste,-,-,-,-,RO,11
1,1100023,11,Ariquemes,-,-,-,-,RO,11
2,1100031,11,Cabixi,-,-,-,-,RO,11
3,1100049,11,Cacoal,Masculino,52,Branca,Ensino superior incompleto,RO,11
4,1100056,11,Cerejeiras,Feminino,63,Parda,Ensino superior completo,RO,11
...,...,...,...,...,...,...,...,...,...
5565,5222005,52,Vianópolis,Feminino,(*) Não soube informar,Branca,Ensino superior completo,GO,52
5566,5222054,52,Vicentinópolis,-,-,-,-,GO,52
5567,5222203,52,Vila Boa,-,-,-,-,GO,52
5568,5222302,52,Vila Propício,-,-,-,-,GO,52


In [8]:
df = df.drop(['Cod UF'], axis=1) #eliminando coluna repetida

In [9]:
df['ano']=2019

In [10]:
df = df.rename(columns={'UF':'sigla_uf',
                        'COD UF':'cod_uf',
                        'CodMun':'id_municipio',
                        'NOME MUNIC':'nome_municipio',
                        'MDHU04':'genero',
                        'MDHU05':'idade',
                        'MDHU06':'cor_raca',
                        'MDHU07':'grau_instrucao'})

In [11]:
#padronizando os dados da coluna 'genero'
df['genero']=np.where(df['genero']=='-','Sem dados',df['genero'])

#padronizando os dados da coluna 'idade'
df['idade']=np.where(df['idade']=='(*) Não soube informar',np.nan,df['idade'])
df['idade']=np.where(df['idade']=='-',np.nan,df['idade'])
df['idade']=np.where(df['idade']=='Recusa',np.nan,df['idade'])
df['idade']=np.where(df['idade']=='Sem titular',np.nan,df['idade']) 

#padronizando os dados da coluna 'cor_raca'

df['cor_raca']=np.where(df['cor_raca']=='-','Sem dados',df['cor_raca'])


#padronizando os dados da coluna 'grau_instrucao'
df['grau_instrucao']=np.where(df['grau_instrucao']=='-','Sem dados',df['grau_instrucao'])

In [12]:
df['idade'] =pd.to_numeric(df['idade']) #transformando em numerico 


In [13]:
limites = [0, 30, 50,65,100]
categorias = ['Entre 18-29', 'Entre 30-49', 'Entre 50-64', 'Acima de 65']

df['faixa_etaria'] = pd.cut(df['idade'], bins=limites, labels=categorias)


In [14]:
df=df[['ano', 'sigla_uf','nome_municipio','id_municipio', 'genero', 'cor_raca', 'grau_instrucao', 'faixa_etaria']] #reorganizando as colunas

In [15]:
# criando dicionário
dict_esco = {'Ensino fundamental incompleto':'Até Ensino Fundamental',
             'Ensino fundamental completo':'Até Ensino Fundamental',
             'Ensino médio incompleto':'Até Ensino Médio',
             'Ensino médio completo':'Até Ensino Médio',
             'Ensino superior incompleto':'Até Ensino Superior Completo',
             'Ensino superior completo':'Até Ensino Superior Completo',
             'Especialização':'Até Pós Graduação ou Mestrado',
             'Mestrado':'Até Pós Graduação ou Mestrado',
             'Doutorado':'Até Doutorado'}


In [16]:
df = df.replace({'grau_instrucao':dict_esco}) #substituindo valores para padronizar

In [19]:
df.columns

Index(['ano', 'sigla_uf', 'nome_municipio', 'id_municipio', 'genero',
       'cor_raca', 'grau_instrucao', 'faixa_etaria'],
      dtype='object')

In [29]:
df['ano'].unique()

array([2019], dtype=int64)

Subindo para o GBQ

In [ ]:
client = bigquery.Client()
dataset_ref = client.dataset('cargos_lideranca')

In [ ]:
schema=[bigquery.SchemaField('ano','INTEGER',description='Ano de referência da observação'),
        bigquery.SchemaField('sigla_uf','STRING',description='Sigla da Unidade da Federação referente municipio'),
        bigquery.SchemaField('nome_municipio','STRING',description='Nome do município'),
        bigquery.SchemaField('id_municipio','INTEGER',description='Código do IBGE do município'),
        bigquery.SchemaField('genero','STRING',description='Gênero autodeclarado ou não'),
        bigquery.SchemaField('cor_raca','STRING',description='Raça/cor da pessoa observada'),
        bigquery.SchemaField('grau_instrucao','STRING',description='Escolaridade da pessoa ou do vínculo observado com detalhamento na pós-graduação'),
        bigquery.SchemaField('faixa_etaria','STRING',description='faixa etária da observação')
        ]


In [ ]:
table_ref = dataset_ref.table('MUNIC_perfil_gestor_direitos_humanos')
job_config = bigquery.LoadJobConfig(schema=schema)
job = client.load_table_from_dataframe(df,table_ref, job_config=job_config)
job.result() 

LoadJob<project=repositoriodedadosgpsp, location=US, id=62603ce1-23b9-44b3-82f0-b47fad031677>